In [1]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("model_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting model_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting model_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting model_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting model_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf

In [3]:
# Choose hyperparameters
learning_rate = 0.01
batch_size = 100
training_iterations = 30
display_step = 2

In [4]:
# graph inputs
x = tf.placeholder("float", [None, 784])  # 28*28 pixels image input
y = tf.placeholder("float", [None, 10])   # 10 classes for 0-9 digits

In [6]:
# setting models weights
w = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [7]:
with tf.name_scope("wx_b") as scope:
    # constructing a linear model
    model = tf.nn.softmax(tf.matmul(x, w) + b)   # softmax

In [14]:
w_h = tf.summary.histogram("weights", w)
b_h = tf.summary.histogram("biases", b)


In [15]:
with tf.name_scope("cost_function") as scope:
    # Minimize the error
    cost_function = -tf.reduce_sum(y*tf.log(model))
    
    tf.summary.scalar("cost function: ", cost_function)

INFO:tensorflow:Summary name cost function:  is illegal; using cost_function__ instead.


In [16]:
with tf.name_scope("train") as scope:
    # Gradient Descent 
    optimiser = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [18]:
init = tf.global_variables_initializer()

In [19]:
merged_summary_ops = tf.summary.merge_all()

In [36]:
with tf.Session() as sess:
    sess.run(init)
    
    summary_writer = tf.summary.FileWriter('/home/ayush/tensorflow/Projects', graph_def = sess.graph_def)
    
    # Training Cycle
    for iteration in range(training_iterations):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        
        # Loop over all branches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            # Fit training using batch data
            sess.run(optimiser, feed_dict={x:batch_xs, y:batch_ys})
            
            # compute average loss
            avg_cost += sess.run(cost_function, feed_dict={x:batch_xs, y:batch_ys})/total_batch
            
            # write log for each iteration
            summary_str = sess.run(merged_summary_ops, feed_dict={x:batch_xs, y:batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch + 1)
            
        # Display log per iteration
        if iteration % display_step == 0:
            print("iteration {}: Cost : {:.9f}".format(iteration+1, avg_cost))
    print('Tunning Completed')
    
# Testing model
predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))

# Accuracy
accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
print("Accuracy : {:.6f}".format(accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels})))


iteration 1: Cost : 30.177527863
iteration 3: Cost : 21.015275117
iteration 5: Cost : 20.125455413
iteration 7: Cost : 19.727219844
iteration 9: Cost : 19.367440875
iteration 11: Cost : 19.056157601
iteration 13: Cost : 18.952463658
iteration 15: Cost : 18.762104859
iteration 17: Cost : 18.623940451
iteration 19: Cost : 18.507635753
iteration 21: Cost : 18.435077528
iteration 23: Cost : 18.297558823
iteration 25: Cost : 18.236771045
iteration 27: Cost : 18.287245323
iteration 29: Cost : 18.135384978
Tunning Completed


ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`